In [1]:
from google.colab import files
uploaded = files.upload()

Saving tr_data.csv to tr_data.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn.preprocessing as skl_pre
#import sklearn.linear_model as skl_lm
import sklearn.discriminant_analysis as skl_da
import sklearn.neighbors as skl_nb

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

In [3]:
df = pd.read_csv('tr_data.csv', na_values='?', dtype={'ID': str}).dropna().reset_index()
print(df)

      index  hour_of_day  day_of_week  month  holiday  weekday  summertime  \
0         0            5            5      1        0        0           0   
1         1           21            4      1        0        1           0   
2         2           21            3      8        0        1           1   
3         3            1            6      1        0        0           0   
4         4           17            0      3        0        1           0   
...     ...          ...          ...    ...      ...      ...         ...   
1595   1595            3            5      6        0        0           1   
1596   1596           14            0      6        0        1           1   
1597   1597           13            0      3        0        1           1   
1598   1598           14            5      3        0        0           1   
1599   1599           22            6      2        0        0           0   

      temp   dew  humidity  precip  snow  snowdepth  windspeed 

In [4]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
df['increase_stock'] = label_encoder.fit_transform(df['increase_stock'])

# Check the mapping of the categories to numbers
label_mapping = dict(zip(label_encoder.classes_,label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'high_bike_demand': 0, 'low_bike_demand': 1}


In [5]:
X = df.drop(columns = ['increase_stock'])
y = df['increase_stock']

print(X.shape,y.shape)

(1600, 16) (1600,)


In [6]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1280, 16) (320, 16) (1280,) (320,)


In [7]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

print("Class distribution after Random Oversampling:", pd.Series(y_train_ros).value_counts())

Class distribution after Random Oversampling: increase_stock
1    1042
0    1042
Name: count, dtype: int64


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
model = LogisticRegression(solver = 'liblinear', max_iter=1000, random_state=42)

#model = LogisticRegression(solver = 'lbfgs', max_iter=1000, random_state=42)

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

err = np.mean(y_pred != y_test)
print("Error", err)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


Error 0.1625
Accuracy: 0.84
              precision    recall  f1-score   support

           0       0.48      0.44      0.46        50
           1       0.90      0.91      0.90       270

    accuracy                           0.84       320
   macro avg       0.69      0.68      0.68       320
weighted avg       0.83      0.84      0.83       320

[[ 22  28]
 [ 24 246]]


In [ ]:
# Define parameter distributions for random search
param_distributions = {
    "C": loguniform(0.001, 100),
    "solver": ["liblinear", "lbfgs"],
    "class_weight": [None, "balanced"],
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=LogisticRegression(random_state=42, max_iter=1000),
    param_distributions=param_distributions,
    n_iter=50,
    cv=5,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1,
)

random_search.fit(X_train_scaled, y_train)

best_params_random = random_search.best_params_
best_score_random = random_search.best_score_

print("Best Parameters (Random Search):", best_params_random)
print("Best Cross-Validation Accuracy (Random Search):", best_score_random)


Best Parameters (Random Search): {'C': 0.061991000078022655, 'class_weight': None, 'solver': 'lbfgs'}
Best Cross-Validation Accuracy (Random Search): 0.85
